##INSTRUCTION:


1.   This program is trained and tested on the GPU environment. Plese make sure that your colab notebook is connected to GPU Runtime.
2.  The main aim of the below network is to observe the major differences between training from scratch and Transfer Learning (using Pre-trained weights).




Student Deatils:

In [1]:
name = 'Sachin Singh'
id = '1141124'
print(f"My name is : {name},\nMy student id : {id}\nMy last digit of ID belongs to : {id[-1]}\nDataset to be selected: Scene-15 dataset and Caltech256")
print(f"My second last digit of ID belongs to : {id[-2]}\nDCNN to be selected: DenseNet ")

My name is : Sachin Singh,
My student id : 1141124
My last digit of ID belongs to : 4
Dataset to be selected: Scene-15 dataset and Caltech256
My second last digit of ID belongs to : 2
DCNN to be selected: DenseNet 


Importing All Important Libraries.

In [1]:
from keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.initializers import he_normal

Importing Time to compare the differences.

In [2]:
from time import time

Data was dowloaded on the local file. Please specify your specific path or used the data submitted along with this code file.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading data, Converting images into RGB, Scalling, Batching

In [3]:
def generator(dir, gen = image.ImageDataGenerator(rescale = 1./255), shuffle = True,
            batch_size = 1, target_size=(24, 24), class_mode = 'categorical'):
    return gen.flow_from_directory(dir,batch_size=batch_size, shuffle=shuffle, color_mode='rgb',
                                   class_mode=class_mode, target_size=target_size)

Defining Batch Size and Target Size

In [4]:
BS = 32
TS = (224, 224)

**Scene-15 dataset** 

Loading Data into Train and Validate batch.

> total no of classes = 15

> total no of images = 4485

Assignment 4 requirements 

> Randomly selecting 100 images per class (15 X 100) = 1500 images for training.

> Remaining images are used for testing (4485 - 1500) = 2985

In [5]:
train_batch = generator('/content/drive/MyDrive/15-Scene/train', shuffle=True, batch_size = BS, target_size = TS)
valid_batch = generator('/content/drive/MyDrive/15-Scene/test', shuffle=True, batch_size = BS, target_size = TS)

Found 1500 images belonging to 15 classes.
Found 2985 images belonging to 15 classes.


Defining Input Shape

In [6]:
initial = he_normal()
input_shape_densenet = (224, 224, 3)

Creating Model object with Weights None. (Training from Scratch).

> When weights None. (Training from Scratch).

> When weights = 'imagnet' (pre-trained weights, Transfer learning)

In [7]:
densenet_model = DenseNet169(
    include_top=False,
    weights= "imagenet",
    input_tensor=None,
    input_shape=input_shape_densenet,
    pooling=None
)

51888128/51877672 [==============================] - 1s 0us/step


In [8]:
# uncomment to see the training from scratch
# densenet_model = DenseNet169(
#     include_top=False,
#     weights= None,
#     input_tensor=None,
#     input_shape=input_shape_densenet,
#     pooling=None
# )

In [9]:
densenet_model.trainable = True
for layer in densenet_model.layers:
  if 'conv5' in layer.name:
    layer.trainable = True
  else:
    layer.trainable = False

In [9]:
# uncomment to see the training from scratch
# densenet_model.trainable = True
# for layer in densenet_model.layers:
#   if 'conv5' in layer.name:
#     layer.trainable = True
#   else:
#     layer.trainable = False

In [10]:
input = Input(input_shape_densenet)
new_layer = densenet_model(input)

Flatining the layer and adding FC layers.

In [11]:
new_layer = Flatten()(new_layer)

new_layer = BatchNormalization()(new_layer)

new_layer = Dense(units=256, activation='relu', kernel_initializer= initial)(new_layer)

In [12]:
new_layer = Dropout(0.4)(new_layer)

new_layer = BatchNormalization()(new_layer)

new_layer = Dense(units=128, activation='relu', kernel_initializer= initial)(new_layer)

Output layer with Softmax Activation funaction.

In [13]:
new_layer = Dropout(0.4)(new_layer)

output = Dense(units=15, activation='softmax', kernel_initializer= initial)(new_layer)

Printing Model Summary.

In [14]:
densenet_model = Model(inputs=input, outputs=output)
densenet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 batch_normalization (BatchN  (None, 81536)            326144    
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               20873472  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

Compiling the model.

In [15]:
densenet_model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

Fitting the model.

This is for training from scratch single Run with 40 epochs.

In [17]:
previous = densenet_model.fit(train_batch, validation_data=valid_batch, epochs=40, verbose=1)

Epoch 1/40
47/47 [==============================] - 788s 16s/step - loss: 0.5110 - accuracy: 0.0953 - val_loss: 0.2656 - val_accuracy: 0.0553
Epoch 2/40
47/47 [==============================] - 66s 1s/step - loss: 0.2770 - accuracy: 0.1807 - val_loss: 0.2568 - val_accuracy: 0.0881
Epoch 3/40
47/47 [==============================] - 66s 1s/step - loss: 0.2306 - accuracy: 0.2987 - val_loss: 0.2626 - val_accuracy: 0.1233
Epoch 4/40
47/47 [==============================] - 66s 1s/step - loss: 0.2052 - accuracy: 0.3893 - val_loss: 0.3721 - val_accuracy: 0.0727
Epoch 5/40
47/47 [==============================] - 66s 1s/step - loss: 0.1854 - accuracy: 0.4260 - val_loss: 0.4008 - val_accuracy: 0.0687
Epoch 6/40
47/47 [==============================] - 66s 1s/step - loss: 0.1645 - accuracy: 0.5073 - val_loss: 0.3321 - val_accuracy: 0.1069
Epoch 7/40
47/47 [==============================] - 66s 1s/step - loss: 0.1456 - accuracy: 0.5593 - val_loss: 0.4147 - val_accuracy: 0.1826
Epoch 8/40
47/47 [

This is for Training from Pretarined
running 3 times and taking average accuracy for each time.

In [18]:
for i in range(1, 4):
  print(f"This is {i} Run for 6 epochs")
  version = densenet_model.fit(train_batch, validation_data=valid_batch, epochs=6, verbose=1)

This is 1 Run for 6 epochs
Epoch 1/6
47/47 [==============================] - 777s 16s/step - loss: 0.3914 - accuracy: 0.3547 - val_loss: 0.2173 - val_accuracy: 0.5059
Epoch 2/6
47/47 [==============================] - 41s 886ms/step - loss: 0.1080 - accuracy: 0.7760 - val_loss: 0.0950 - val_accuracy: 0.7481
Epoch 3/6
47/47 [==============================] - 41s 887ms/step - loss: 0.0645 - accuracy: 0.8980 - val_loss: 0.0555 - val_accuracy: 0.8590
Epoch 4/6
47/47 [==============================] - 41s 886ms/step - loss: 0.0423 - accuracy: 0.9347 - val_loss: 0.0506 - val_accuracy: 0.8707
Epoch 5/6
47/47 [==============================] - 41s 885ms/step - loss: 0.0292 - accuracy: 0.9640 - val_loss: 0.0431 - val_accuracy: 0.8911
Epoch 6/6
47/47 [==============================] - 41s 886ms/step - loss: 0.0212 - accuracy: 0.9747 - val_loss: 0.0443 - val_accuracy: 0.8827
This is 2 Run for 6 epochs
Epoch 1/6
47/47 [==============================] - 41s 889ms/step - loss: 0.0164 - accuracy: 0.